### NHL-Game Part 2 Historical and New Data Storage 🥅 🏒

In [0]:
# File location and type
file_location = "/FileStore/tables/Period_1_Game_Stats_Final_ModelReady.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
nhl_db = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)



In [0]:
# Importing correct packages
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

# Machine Learning Libraries
import lightgbm as lgb
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Figures
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="whitegrid")

In [0]:
nhl_db=nhl_db.toPandas()

In [0]:
# convert from spark to pandas dataframe format
df = nhl_db

In [0]:
df.shape


Out[9]: (51558, 21)

In [0]:
# No null values

df = df[:-9611]
df.drop(df.index[18800:40871],0,inplace=True)
df = df.reset_index(drop=True)
df.isna().sum()

/databricks/python/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
Out[10]: won                      0
Shots                    0
Shots_Against            0
Goals                    0
Goals_Against            0
Takeaways                0
Takeaways_Against        0
Hits                     0
Hits_Against             0
Blocked Shots            0
Blocked Shots Against    0
Giveaways                0
Giveaways_Against        0
Missed Shots             0
Missed Shots_Against     0
Penalities               0
Penalities_Against       0
#Won Faceoffs            0
#Lost Faceoffs           0
HoA_away                 0
HoA_home                 0
dtype: int64

In [0]:
df.head(5)

,won,Shots,Shots_Against,Goals,Goals_Against,Takeaways,Takeaways_Against,Hits,Hits_Against,Blocked Shots,Blocked Shots Against,Giveaways,Giveaways_Against,Missed Shots,Missed Shots_Against,Penalities,Penalities_Against,#Won Faceoffs,#Lost Faceoffs,HoA_away,HoA_home
0,0,8.0,8.0,0.0,3.0,1.0,3.0,14.0,5.0,3.0,3.0,6.0,7.0,4.0,0.0,1.0,1.0,10.0,12.0,1,0
1,1,8.0,8.0,3.0,0.0,3.0,1.0,5.0,14.0,3.0,3.0,7.0,6.0,0.0,4.0,1.0,1.0,12.0,10.0,0,1
2,1,11.0,12.0,0.0,1.0,0.0,2.0,4.0,4.0,6.0,8.0,2.0,0.0,6.0,3.0,1.0,3.0,11.0,9.0,1,0
3,0,12.0,11.0,1.0,0.0,2.0,0.0,4.0,4.0,8.0,6.0,0.0,2.0,3.0,6.0,3.0,1.0,9.0,11.0,0,1
4,1,9.0,8.0,0.0,1.0,3.0,2.0,4.0,5.0,7.0,1.0,7.0,5.0,3.0,8.0,2.0,1.0,7.0,11.0,1,0


In [0]:
# split data into X and y
y = df.pop('won')
X = df
# split data into train and test sets
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

In [0]:
model = lgb.LGBMClassifier(learning_rate=0.09,max_depth=-5,random_state=42)
model.fit(X_train,y_train,eval_set=[(X_test,y_test),(X_train,y_train)],
          verbose=20,eval_metric='logloss')

/databricks/python/lib/python3.8/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
[20]	training's binary_logloss: 0.588145	valid_0's binary_logloss: 0.602557
[40]	training's binary_logloss: 0.564346	valid_0's binary_logloss: 0.599359
[60]	training's binary_logloss: 0.546199	valid_0's binary_logloss: 0.600596
[80]	training's binary_logloss: 0.529762	valid_0's binary_logloss: 0.60314
[100]	training's binary_logloss: 0.514817	valid_0's binary_logloss: 0.604746
Out[13]: LGBMClassifier(learning_rate=0.09, max_depth=-5, random_state=42)

In [0]:
print('Training accuracy {:.4f}'.format(model.score(X_train,y_train)))
print('Testing accuracy {:.4f}'.format(model.score(X_test,y_test)))

Training accuracy 0.7570
Testing accuracy 0.6538


In [0]:
# Interestingly, slightly better accuracy
# make predictions for test data
from sklearn.metrics import precision_score, recall_score
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print('Precision score: ', (precision_score(y_test, predictions)))
print('Recall score: ', recall_score(y_test, predictions))

Accuracy: 65.38%
Precision score:  0.6565500154846702
Recall score:  0.6459475929311396


In [0]:
# make predictions for tonight data (optimized for continuous prediciton)
def tonight_bet(*paramas):
  shots=paramas[0]
  shots_agianst=paramas[1]
  goal=paramas[2]
  goal_against=paramas[3]
  takeaway=paramas[4]
  takeaway_against=paramas[5]
  hits=paramas[6]
  hits_against=paramas[7]
  blockedShots=paramas[8]
  blockedShots_against=paramas[9]
  giveaway=paramas[10]
  giveaway_against=paramas[11]
  missedShots=paramas[12]
  missedShotsagainst=paramas[13]
  penalties=paramas[14]
  penalties_against=paramas[15]
  Won_Faceoffs=paramas[16]
  Lost_Faceoffs=paramas[17]
  hoa_away=paramas[18]
  hoa_home=paramas[19]
  bet_df=pd.DataFrame({"shots":[shots],"shots_agianst":[shots_agianst],"goal":[goal],"goal_against":[goal_against],"takeaway":[takeaway],
                       "takeaway_against":[takeaway_against],"hits":[hits],"hits_against":[hits_against],"blockedShots":[blockedShots],
                       "blockedShots_against":[blockedShots_against],"giveaway":[giveaway],"giveaway_against":[giveaway_against],"missedShots":[missedShots],
                       "missedShotsagainst":[missedShotsagainst],"penalties":[penalties],"penalties_against":[penalties_against],"Won_Faceoffs":[Won_Faceoffs],
                       "Lost_Faceoffs":[Lost_Faceoffs],"hoa_away":[hoa_away],"hoa_home":[hoa_home]})
  bet_df=bet_df.iloc[:1,:]
  tonight_teams=bet_df
  return tonight_teams

In [0]:
x=[34,5,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
tonight_bet(*x)



,shots,shots_agianst,goal,goal_against,takeaway,takeaway_against,hits,hits_against,blockedShots,blockedShots_against,giveaway,giveaway_against,missedShots,missedShotsagainst,penalties,penalties_against,Won_Faceoffs,Lost_Faceoffs,hoa_away,hoa_home
0,34,5,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
# make predictions for test data
X_Game_test = tonight_bet(*x)
# store value to use in model

Game_y_pred = model.predict(X_Game_test)
prediction = [round(value) for value in Game_y_pred]
print(prediction)
probability = model.predict_proba(X_Game_test)
print(probability)

[1]
[[0.23307611 0.76692389]]


# API SportsRadar Request

In [0]:
# API Packages
import requests
import json
api_key = "sa8b3wuwy549qwy6jw6srsgv"
print("API Key:",api_key)

API Key: sa8b3wuwy549qwy6jw6srsgv


In [0]:
from datetime import date
import re
from datetime import datetime,timedelta

today = date.today()

# dd/mm/YY
Day = today.strftime("%d")
Month = today.strftime("%m")
Year = today.strftime("%Y")
today = date.today()
print("Today's date:", today)
# Get API info
response = requests.get(("https://api.sportradar.us/nhl/trial/v7/en/games/{Year}/{Month}/{Day}/schedule.json?api_key=sa8b3wuwy549qwy6jw6srsgv").format(Day=Day, Month = Month, Year = Year))

# Define function that translates API into text
def jprint(obj):
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)
    return ""

# Set response as json
x = response.json()
# Print API info
def getrs(obj):
    Todays_Schedule = []
    Todays_Schedule = pd.DataFrame([],columns = ["Game ID","Home Team",'Home Team ID',"Away Team","Away Team ID","Start Time"])
    q = len(obj["games"])
    for i in range(q):
            GID = obj["games"][i]["id"]
            H = obj['games'][i]['home']['name']
            Hid = obj['games'][i]['home']['id']
            A = obj['games'][i]['away']['name']
            Aid = obj['games'][i]['away']['id']
            PD = obj['games'][i]['scheduled']
            PD = re.search('T(.+?)Z',PD)
            if PD:
                found = PD.group(1)
            format = "%H:%M:%S"
            d = datetime.strptime(found,format) - timedelta(hours=4)
            d = d.strftime("%H:%M:%S")
            PD = d
            temp_df = [GID,H,Hid,A,Aid,PD]
            a_series = pd.Series(temp_df,index = Todays_Schedule.columns)
            Todays_Schedule = Todays_Schedule.append(a_series, ignore_index=True)
    return Todays_Schedule
data=getrs(x)
data.head(10)

Today's date: 2022-04-16


,Game ID,Home Team,Home Team ID,Away Team,Away Team ID,Start Time
0,305e7c9a-fff5-453c-a3d7-5f9ea2b026d4,Boston Bruins,4416ba1a-0f24-11e2-8525-18a905767e44,Pittsburgh Penguins,4417b7d7-0f24-11e2-8525-18a905767e44,12:30:00
1,a6110d51-b977-4503-bb03-d46cbe5d4840,New York Rangers,441781b9-0f24-11e2-8525-18a905767e44,Detroit Red Wings,44169bb9-0f24-11e2-8525-18a905767e44,12:30:00
2,df355913-7846-4eeb-8ad4-528dca0077d6,Nashville Predators,441643b7-0f24-11e2-8525-18a905767e44,Chicago Blackhawks,4416272f-0f24-11e2-8525-18a905767e44,12:30:00
3,09ebf3f9-d2e0-4ac5-b2ab-550e5a90ddce,St. Louis Blues,441660ea-0f24-11e2-8525-18a905767e44,Minnesota Wild,4416091c-0f24-11e2-8525-18a905767e44,15:00:00
4,37425f01-d778-432b-ad37-d5c49282f395,Edmonton Oilers,4415ea6c-0f24-11e2-8525-18a905767e44,Vegas Golden Knights,42376e1c-6da8-461e-9443-cfcf0a9fcc4d,16:00:00
5,3aa3ada8-fd91-4dee-a01d-39b35046355b,Ottawa Senators,4416f5e2-0f24-11e2-8525-18a905767e44,Toronto Maple Leafs,441730a9-0f24-11e2-8525-18a905767e44,19:00:00
6,63f102bd-42ed-45b0-bb74-44dd0f6ab0a5,Buffalo Sabres,4416d559-0f24-11e2-8525-18a905767e44,Philadelphia Flyers,44179d47-0f24-11e2-8525-18a905767e44,19:00:00
7,9511fcc4-6851-4cac-8f84-7306000d0bea,Montreal Canadiens,441713b7-0f24-11e2-8525-18a905767e44,Washington Capitals,4417eede-0f24-11e2-8525-18a905767e44,19:00:00
8,b36484be-e914-4ab1-b4ac-23f70306c355,Tampa Bay Lightning,4417d3cb-0f24-11e2-8525-18a905767e44,Winnipeg Jets,44180e55-0f24-11e2-8525-18a905767e44,19:00:00
9,31d0a5c3-4d22-498f-a338-7004cf1a4580,Dallas Stars,44157522-0f24-11e2-8525-18a905767e44,San Jose Sharks,44155909-0f24-11e2-8525-18a905767e44,20:00:00


# Current Game Stats

In [0]:
def Game_Home_Team_Stats(game_id,home_team_id):
    ts = 0
    tsa = 0
    go = 0
    goa = 0
    t = 0
    ta = 0
    h = 0
    ha = 0
    bs = 0
    bsa = 0
    gi = 0
    gia = 0
    ms = 0
    msa = 0
    p = 0
    pa = 0
    tfw = 0
    tfl = 0
    away = 0
    home = 0
    response = requests.get(("https://api.sportradar.us/nhl/trial/v7/en/games/{obj}/pbp.json?api_key=sa8b3wuwy549qwy6jw6srsgv").format(obj = game_id))
    x = response.json()
    y = len(x['periods'][0]['events'])
    htc = x['home']['id']
    if htc == home_team_id:
        home = 1
    else:
        away = 1
    for i in range(y):
        j = x['periods'][0]['events'][i]['event_type']
        if j == 'faceoff':
            team_id = x['periods'][0]['events'][i]['attribution']['id']
            if team_id == home_team_id:
                    tfw = tfw + 1
            else:
                    tfl = tfl + 1
        elif j == 'goal':
            team_id = x['periods'][0]['events'][i]['attribution']['id']
            if team_id == home_team_id:
                    go = go + 1
            else:
                    goa = goa + 1
        elif j == 'penalty':
            team_id = x['periods'][0]['events'][i]['attribution']['id']
            if team_id == home_team_id:
                    p = p + 1
            else:
                    pa = pa + 1
        elif j == 'shotmissed' and 'blocked' in x['periods'][0]['events'][i]['description']:
                team_id = x['periods'][0]['events'][i]['attribution']['id']
                if team_id == home_team_id:
                        bs = bs + 1
                else:
                        bsa = bsa + 1
        elif j == 'shotmissed' and 'missed' in x['periods'][0]['events'][i]['description']:
                team_id = x['periods'][0]['events'][i]['attribution']['id']
                if team_id == home_team_id:
                        ms = ms + 1
                else:
                        msa = msa + 1
        elif j == 'giveaway':
            team_id = x['periods'][0]['events'][i]['attribution']['id']
            if team_id == home_team_id:
                    gi = gi + 1
            else:
                    gia = gia + 1
        elif j == 'hit':
            team_id = x['periods'][0]['events'][i]['attribution']['id']
            if team_id == home_team_id:
                    h = h + 1
            else:
                    ha = ha + 1
        elif j == 'takeaway':
            team_id = x['periods'][0]['events'][i]['attribution']['id']
            if team_id == home_team_id:
                    t = t + 1
            else:
                    ta = ta + 1
        elif 'shot' in j:
            team_id = x['periods'][0]['events'][i]['attribution']['id']
            if team_id == home_team_id:
                    ts = ts + 1
            else:
                    tsa = tsa + 1
        else:
            pass
    d = {"shots": [ts], "shots_against": [tsa],"goals":[go],"goals_against":[goa],"takeaways":[t],"takeaways_against":[ta],"hits":[h],"hits_against":[ha],"blockedShots":[bs],"blockedShots_against":[bsa],"giveaways":[gi],"giveaways_against":[gia],"missedShots":[ms],"missedShots_against":[msa],"penalties":[p],"penalties_against":[pa],"#Won Faceoffs":[tfw],"#Lost Faceoffs":[tfl],'HoA_away':[away],'HoA_home':[home]}
    Home_Team_Stats = pd.DataFrame(d)
    return Home_Team_Stats
 

Game_Home_Team_Stats('c134d1ff-1c8f-4591-9707-452010c9597d','44157522-0f24-11e2-8525-18a905767e44')

,shots,shots_against,goals,goals_against,takeaways,takeaways_against,hits,hits_against,blockedShots,blockedShots_against,giveaways,giveaways_against,missedShots,missedShots_against,penalties,penalties_against,#Won Faceoffs,#Lost Faceoffs,HoA_away,HoA_home
0,9,7,0,0,1,1,11,3,4,1,2,6,9,2,1,1,11,4,0,1


In [0]:
# new observations
game= Game_Home_Team_Stats('2f0a0e46-7248-49bb-a4ab-cd21c59b0436','4417eede-0f24-11e2-8525-18a905767e44')

In [0]:
print(game)
type(game)

   shots  shots_against  goals  goals_against  takeaways  takeaways_against  \
0      7              7      3              1          7                  0   

   hits  hits_against  blockedShots  blockedShots_against  giveaways  \
0     8             6             5                     4          2   

   giveaways_against  missedShots  missedShots_against  penalties  \
0                  2            5                    6          1   

   penalties_against  #Won Faceoffs  #Lost Faceoffs  HoA_away  HoA_home  
0                  1             11               8         0         1  
Out[28]: pandas.core.frame.DataFrame

In [0]:
stats= game.iloc[0,:]
stats_list= stats.tolist()


In [0]:
# make predictions for test data
X_Game_test = tonight_bet(*stats_list)
# store value to use in model

Game_y_pred = model.predict(X_Game_test)
prediction = [round(value) for value in Game_y_pred]
print(prediction)
probability = model.predict_proba(X_Game_test)
print(probability)

[1]
[[0.4030312 0.5969688]]


In [0]:
type(stats_list)
len(stats_list)

Out[32]: 20

In [0]:
# add the new observation info
for i in prediction:
    stats_list.append(i)
print(stats_list)
len(stats_list)

[7, 7, 3, 1, 7, 0, 8, 6, 5, 4, 2, 2, 5, 6, 1, 1, 11, 8, 0, 1, 1]
Out[30]: 21

In [0]:
nhl_db = nhl_db.append(pd.DataFrame([stats_list], columns=nhl_db.columns), ignore_index=True)


In [0]:
nhl_db.shape

Out[32]: (51559, 21)

In [0]:
# convert pandas to spark format and create delta lake 
nhl_db_sp= spark.createDataFrame(nhl_db) 
#sparkDF=spark.createDataFrame(pandasDF) 



In [0]:
type(nhl_db_sp)

Out[34]: pyspark.sql.dataframe.DataFrame

In [0]:
nhl_db_sp.show(10)

+---+-----+-------------+-----+-------------+---------+-----------------+----+------------+-------------+---------------------+---------+-----------------+------------+--------------------+----------+------------------+-------------+--------------+--------+--------+
|won|Shots|Shots_Against|Goals|Goals_Against|Takeaways|Takeaways_Against|Hits|Hits_Against|Blocked Shots|Blocked Shots Against|Giveaways|Giveaways_Against|Missed Shots|Missed Shots_Against|Penalities|Penalities_Against|#Won Faceoffs|#Lost Faceoffs|HoA_away|HoA_home|
+---+-----+-------------+-----+-------------+---------+-----------------+----+------------+-------------+---------------------+---------+-----------------+------------+--------------------+----------+------------------+-------------+--------------+--------+--------+
|  0|  8.0|          8.0|  0.0|          3.0|      1.0|              3.0|14.0|         5.0|          3.0|                  3.0|      6.0|              7.0|         4.0|                 0.0|       1.0

In [1]:
## Set Up Delta Lake 

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import *

In [0]:
%sql
create database if not exists nhl_delta_lake

In [0]:
nhl_db_sp= nhl_db_sp.withColumnRenamed("Takeaways Against", "TakeawaysAgainst")
nhl_db_sp= nhl_db_sp.withColumnRenamed("#Won Faceoffs", "WonFaceoffs")
nhl_db_sp= nhl_db_sp.withColumnRenamed("#Lost Faceoffs", "LostFaceoffs")
nhl_db_sp= nhl_db_sp.withColumnRenamed("Shots Against", "ShotsAgainst")
nhl_db_sp= nhl_db_sp.withColumnRenamed("Goals Against", "GoalsAgainst")
nhl_db_sp= nhl_db_sp.withColumnRenamed("Hits gainst", "HitsAgainst")
nhl_db_sp= nhl_db_sp.withColumnRenamed("Giveaways_Against", "GivewaysAgainst")

nhl_db_sp= nhl_db_sp.withColumnRenamed("Missed Shots Against", "MissedShotsAgainst")
nhl_db_sp= nhl_db_sp.withColumnRenamed("Penalities Against", "PenalitiesAgainst")
nhl_db_sp= nhl_db_sp.withColumnRenamed("HOA away", "HOAaway")
nhl_db_sp= nhl_db_sp.withColumnRenamed("HOA home", "HOAhome")


In [0]:
nhl_db_sp.show(5)

+---+-----+-------------+-----+-------------+---------+-----------------+----+------------+-------------+---------------------+---------+-----------------+------------+--------------------+----------+------------------+-------------+--------------+--------+--------+
|won|Shots|Shots_Against|Goals|Goals_Against|Takeaways|Takeaways_Against|Hits|Hits_Against|Blocked Shots|Blocked Shots Against|Giveaways|Giveaways_Against|Missed Shots|Missed Shots_Against|Penalities|Penalities_Against|#Won Faceoffs|#Lost Faceoffs|HoA_away|HoA_home|
+---+-----+-------------+-----+-------------+---------+-----------------+----+------------+-------------+---------------------+---------+-----------------+------------+--------------------+----------+------------------+-------------+--------------+--------+--------+
|  0|  8.0|          8.0|  0.0|          3.0|      1.0|              3.0|14.0|         5.0|          3.0|                  3.0|      6.0|              7.0|         4.0|                 0.0|       1.0

In [0]:
# create table


#nhl_db.write.format("delta").saveAsTable("nhl_delta_lake")